<a href="https://colab.research.google.com/github/liangli217/LLM_learning/blob/main/LLM_for_genomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DOWNLOAD Mistral-DNA GIT REPO
!git clone https://github.com/raphaelmourad/Mistral-DNA.git

Cloning into 'Mistral-DNA'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 331 (delta 27), reused 30 (delta 5), pack-reused 235 (from 1)
Receiving objects: 100% (331/331), 112.69 MiB | 19.66 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# WHEN USING GOOGLE COLLAB
!pip install datasets==3.0.1
!pip install flash-attn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x

In [ ]:
# CHECK GPU
# We can see how many VRAM is used and how much the GPU is used.
!nvidia-smi

In [ ]:
# IMPORT LIBRARIES
import os

import torch
import numpy as np

from transformers import AutoTokenizer
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoConfig
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset

In [ ]:
# SET DIRECTORY
os.chdir("Mistral-DNA/")
print(os.getcwd())

/content/Mistral-DNA


In [43]:
!ls

data  logs  README.md  results	scriptPython  scriptR


In [ ]:
# CHOOSE THE LLM ARCHITECTURE
# To do during class:
# - look at the original archicture of Mixtral-8x7B-v0.1, discuss the model
# - change the model architecture by adding or removing transformer blocks, hidden states, number of attention heads, and number of experts
# - test BERT model architecture?
# NB: flash attention 2 does not work with T4 GPU
config = AutoConfig.from_pretrained("data/models/Mixtral-8x7B-v0.1") # Mixture of expert
#model = AutoModelForCausalLM.from_config(config,attn_implementation="flash_attention_2")
model = AutoModelForCausalLM.from_config(config,attn_implementation="eager")
model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(4096, 256)
    (layers): ModuleList(
      (0-7): 8 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=256, bias=False)
          (v_proj): Linear(in_features=256, out_features=256, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=256, out_features=64, bias=False)
          (experts): ModuleList(
            (0-63): 64 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=256, out_features=256, bias=False)
              (w2): Linear(in_features=256, out_features=256, bias=False)
              (w3): Linear(in_features=256, out_features=256, bias=False)
              (act_fn): SiLU()
            )
          )
        )
        (

In [ ]:
# LOAD BPE LETTER TOKENIZER
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
tokenizer.padding_side  = 'left'
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='zhihan1996/DNABERT-2-117M', vocab_size=4096, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [ ]:
# DNA encoding
encoidng = tokenizer("ATTGTGGGTCCCCC", padding = "longest", truncation = True, return_tensors = "pt")
print(encoidng)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[   1, 2061,  281,  485,    6,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Model size: {pytorch_total_params/1000**2:.1f}M parameters")

Model size: 105.0M parameters


In [ ]:
# Load data
data_collator = DataCollatorForLanguageModeling(tokenizer= tokenizer, mlm = False)

# The file contains 100,000 non-overlapping DNA sequences of 200 bases.

dataset_text = load_dataset("csv", data_files="data/genome_sequences/hg38/sequences_hg38_200b_verysmall.csv.gz")


In [ ]:
tokenized_data = tokenizer(dataset_text['train']['text'], padding="longest", truncation=True, return_tensors="pt")


In [ ]:
dataset_text['train']

Dataset({
    features: ['text'],
    num_rows: 99999
})

In [ ]:
dataset_text

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 99999
    })
})

In [ ]:
# TOKENIZE DATA
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="longest", truncation=True, return_tensors="pt")

dataset = dataset_text.map(tokenize_function, batched=True)
# print(dataset["train"])

In [ ]:
train_size = int(0.8 * len(dataset["train"]))
test_size = len(dataset["train"]) - train_size
train_set, val_set = torch.utils.data.random_split(dataset['train'], [train_size, test_size])

FP16 vs BF16

In [19]:
# Parameters for Pretraining
batchsize = 32
training_args = TrainingArguments(
    output_dir = './results/models',
    eval_strategy ='epoch',
    save_strategy = 'epoch',
    num_train_epochs = 5,
    per_device_train_batch_size = batchsize,
    per_device_eval_batch_size = batchsize,
    learning_rate = 5e-4,
    weight_decay = 0.01,
    logging_dir = './logs',
    load_best_model_at_end = True,
    bf16 = True,
    gradient_accumulation_steps = 50,
)
import os
os.environ["WANDB_DISABLED"] = "true"

print(training_args)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False,

In [ ]:
!pip install wandb

In [20]:
# Pretrain Model
trainer = Trainer(
    model = model,
    args=training_args,
    data_collator = data_collator,
    train_dataset = train_set,
    eval_dataset = val_set,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

print('Starting a trainer...')
# Start training
trainer.train()

Starting a trainer...


Epoch,Training Loss,Validation Loss
1,No log,6.718481
2,No log,6.649852
3,No log,6.579378
4,No log,6.516723
5,No log,6.496838


TrainOutput(global_step=250, training_loss=6.67015625, metrics={'train_runtime': 7104.908, 'train_samples_per_second': 56.298, 'train_steps_per_second': 0.035, 'total_flos': 1.3184106103312896e+16, 'train_loss': 6.67015625, 'epoch': 5.0})

In [22]:
model

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(4096, 256)
    (layers): ModuleList(
      (0-7): 8 x MixtralDecoderLayer(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=256, out_features=256, bias=False)
          (k_proj): Linear(in_features=256, out_features=256, bias=False)
          (v_proj): Linear(in_features=256, out_features=256, bias=False)
          (o_proj): Linear(in_features=256, out_features=256, bias=False)
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=256, out_features=64, bias=False)
          (experts): ModuleList(
            (0-63): 64 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=256, out_features=256, bias=False)
              (w2): Linear(in_features=256, out_features=256, bias=False)
              (w3): Linear(in_features=256, out_features=256, bias=False)
              (act_fn): SiLU()
            )
          )
        )
        (

In [23]:
model.push_to_hub("simple_genomics_model_first_attempt")

model.safetensors:   0%|          | 0.00/210M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/liangli217/simple_genomics_model_first_attempt/commit/199088c2ab484d71d45478542d2490d5173b3371', commit_message='Upload MixtralForCausalLM', commit_description='', oid='199088c2ab484d71d45478542d2490d5173b3371', pr_url=None, repo_url=RepoUrl('https://huggingface.co/liangli217/simple_genomics_model_first_attempt', endpoint='https://huggingface.co', repo_type='model', repo_id='liangli217/simple_genomics_model_first_attempt'), pr_revision=None, pr_num=None)

In [25]:
# COMPUTE SEQUENCE EMBEDDING
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
inputs = inputs.to(model.device)  # Assuming model is on the GPU

hidden_states = model(inputs)[0] # [1, sequence_length, 4096]
print(hidden_states)

# embedding with mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0)
print(embedding_mean.shape) # expect to be 4096
print(embedding_mean)

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max.shape) # expect to be 4096
print(embedding_max)

tensor([[[-2.4219,  1.7734,  1.4297,  ..., -2.2812, -2.4219, -1.9141],
         [-3.6719, -0.1455, -1.3516,  ..., -3.0000, -2.5781, -2.3594],
         [-3.5156, -0.2988, -1.4453,  ..., -3.0312, -2.4844, -2.3281],
         ...,
         [-3.5000,  0.6680,  0.6250,  ..., -2.8438, -2.5156, -2.3906],
         [-2.5156, -0.3066,  0.7148,  ..., -2.0000, -1.6328, -1.3984],
         [-1.2891,  9.6875,  2.2812,  ..., -1.4688, -1.3750, -1.0312]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
torch.Size([4096])
tensor([-3.2027,  0.8552,  0.1634,  ..., -2.7399, -2.4026, -2.1227],
       device='cuda:0', grad_fn=<MeanBackward1>)
torch.Size([4096])
tensor([-1.2891,  9.6875,  2.2812,  ..., -1.4688, -1.3750, -1.0312],
       device='cuda:0', grad_fn=<MaxBackward0>)


In [33]:
embedding_mean

tensor([-3.2027,  0.8552,  0.1634,  ..., -2.7399, -2.4026, -2.1227],
       device='cuda:0', grad_fn=<MeanBackward1>)

In [32]:
embedding_max

tensor([-1.2891,  9.6875,  2.2812,  ..., -1.4688, -1.3750, -1.0312],
       device='cuda:0', grad_fn=<MaxBackward0>)

In [34]:
dna_wt= "ATTAAAGAAAATATCATCTTTGGTGTTTCCTAT"

In [35]:
dna_mut="ATTAAAGAAAATATCATTGGTGTTTCCTAT"

In [37]:
# COMPUTE THE EMBEDDING OF BOTH DNA SEQUENCES
inputs_seqs = tokenizer([dna_wt,dna_mut], return_tensors = 'pt', padding=True)["input_ids"]
inputs_seqs = inputs_seqs.to(model.device)  # Assuming model is on the GPU

hidden_states_seqs = model(inputs_seqs)[0].detach() # [1, sequence_length, 768]

embedding_max = torch.max(hidden_states_seqs, dim=1)[0]
print(embedding_max)

tensor([[-1.2891,  9.7500,  2.0312,  ..., -1.4141, -1.3594, -1.0156],
        [-0.0884, 10.0000,  2.0000,  ..., -0.3047, -0.3477, -0.3398]],
       device='cuda:0')


In [38]:
# COMPUTE MUCOVISCIDOSE MUTATION EFFECT (GENE CFTR)
distL2=torch.norm(embedding_max[0]-embedding_max[1])
print(distL2) # Mutation effect (unnormalized)

tensor(58.1931, device='cuda:0')


In [51]:
# COMPARE WITH A MUTATION THAT DOES NOT IMPACT THE CFTR PROTEIN SEQUENCE
dna_wt= "ATTAAAGAAAATATCATCTTTGGTGTTTCCTAT"
dna_mut="ATAAAAGAAAATATCATCTTTGGTGTTTCCTAT"

inputs_seqs = tokenizer([dna_wt,dna_mut], return_tensors = 'pt', padding=True)["input_ids"]
inputs_seqs = inputs_seqs.to(model.device)  # Assuming model is on the GPU
hidden_states_seqs = model(inputs_seqs)[0].detach() # [1, sequence_length, 768]

embedding_max = torch.max(hidden_states_seqs, dim=1)[0]
print(embedding_max)

distL2=torch.norm(embedding_max[0]-embedding_max[1])
print(distL2) # Mutation effect (unnormalized)

tensor([[-1.2891,  9.7500,  2.0312,  ..., -1.4141, -1.3594, -1.0156],
        [-1.2969,  9.7500,  2.0312,  ..., -1.4219, -1.3672, -1.0156]],
       device='cuda:0')
tensor(3.2837, device='cuda:0')
